In [17]:
!git clone https://github.com/huggingface/transformers;
!cd transformers; pip3 install .

fatal: destination path 'transformers' already exists and is not an empty directory.
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.12.0.dev0-py3-none-any.whl size=2993464 sha256=0bd9dbcba3a69dc1f92d7dc1ad7e1862b672fb96b2996db5cbb1fe97948fbd93
  Stored in directory: /tmp/pip-ephem-wheel-cache-q78zhcn2/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uninstall: transfor

In [18]:
!pip install -U sentence-transformers

In [19]:
!pip install sacrebleu

In [20]:
!pip install nltk==3.5

Leitura Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
from sentence_transformers import SentenceTransformer, util

import pandas as pd

In [3]:
arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_ptt5.xlsx'
pira_t5 = pd.read_excel(arq)

arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_sinonimo.xlsx'
pira_syn = pd.read_excel(arq)

arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_ppdb.xlsx'
pira_ppdb = pd.read_excel(arq)

arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_ppdb_frasal.xlsx'
pira_frasal = pd.read_excel(arq)

arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_tradutor.xlsx'
pira_trad = pd.read_excel(arq)

arq = '/content/drive/My Drive/Colab Notebooks/assin/assin-parafraseada_portugues_paranet.xlsx'
pira_paranet = pd.read_excel(arq)

In [4]:
pira_syn.head()

,Unnamed: 0,original,label,parafrase
0,0,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",0,"A Lenovo afirma, que o Yoga 900 tem uma bateri..."
1,1,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",0,"A Lenovo afirma, que o Yoga 900 tem uma bateri..."
2,2,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",0,"A Lenovo afirma, que o Yoga 900 tem uma bateri..."
3,3,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",0,"A Lenovo afirma, que o Yoga 900 tem uma pilhas..."
4,4,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",0,"A Lenovo afirma, que domínio Yoga 900 tem uma ..."


In [5]:

pira_t5.columns = ['index', 'original', 'parafrase_t5']


pira_syn = pira_syn[['Unnamed: 0', 'parafrase']]
pira_syn.columns = ['index', 'parafrase_sin']

pira_ppdb = pira_ppdb[['Unnamed: 0', 'parafrase']]
pira_ppdb.columns = ['index', 'parafrase_ppdb']

pira_frasal = pira_frasal[['Unnamed: 0', 'parafrase']]
pira_frasal.columns = ['index', 'parafrase_frasal']

pira_trad = pira_trad[['Unnamed: 0', 'parafrase']]
pira_trad.columns = ['index', 'parafrase_tradutor']

pira_paranet = pira_paranet[['Unnamed: 0', 'parafrase']]
pira_paranet.columns = ['index', 'parafrase_paranet']

In [6]:
pira = pira_t5.merge(pira_syn, on = 'index', how = 'left')
pira = pira.merge(pira_ppdb, on = 'index', how = 'left')
pira = pira.merge(pira_frasal, on = 'index', how = 'left')
pira = pira.merge(pira_trad, on = 'index', how = 'left')
pira = pira.merge(pira_paranet, on = 'index', how = 'left')

In [7]:
for col in pira.columns:
  pira[col] = pira[col].astype('str')

In [22]:
pira.head()

,original,parafrase_t5,parafrase_sin,parafrase_ppdb,parafrase_frasal,parafrase_tradutor,parafrase_paranet,similiaridade_parafrase_t5,similiaridade_parafrase_sin,similiaridade_parafrase_ppdb,similiaridade_parafrase_frasal,similiaridade_parafrase_trad,similiaridade_parafrase_paranet,BLEUscore_t5,BLEUscore_sin,BLEUscore_ppdb,BLEUscore_frasal,BLEUscore_trad,BLEUscore_paranet,BLEUscore_without_bp_t5,BLEUscore_without_bp_sin,BLEUscore_without_bp_ppdb,BLEUscore_without_bp_frasal,BLEUscore_without_bp_trad,BLEUscore_without_bp_paranet
0,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo diz que o Yoga 900 tem uma bateria qu...,"A Lenovo afirma, que o Yoga 900 tem uma bateri...","A Lenovo afirma, deverá o Yoga 900 disponham n...","A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,A Lenovo afirma que o Yoga 900 tem uma pilha q...,0.998639,0.994543,0.932816,1.0,0.974700,0.881348,BLEU = 77.84 91.7/82.6/72.7/66.7 (BP = 1.000 r...,BLEU = 84.83 95.8/91.3/86.4/81.0 (BP = 0.959 r...,BLEU = 36.00 70.8/52.2/31.8/14.3 (BP = 1.000 r...,BLEU = 100.00 100.0/100.0/100.0/100.0 (BP = 1....,BLEU = 63.43 91.7/73.9/63.6/52.4 (BP = 0.920 r...,BLEU = 45.61 83.3/56.5/45.5/33.3 (BP = 0.882 r...,77.841431,88.438659,36.001210,100.0,68.937069,51.685960
1,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo diz que o Yoga 900 tem uma bateria ma...,"A Lenovo afirma, que o Yoga 900 tem uma bateri...","A Lenovo indica, relativamente o Yoga 900 tem ...","A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,A Lenovo afirma que o Yoga 900 tem uma pilhas ...,0.958856,0.993600,0.972226,1.0,0.974700,0.954446,BLEU = 46.67 66.7/52.2/40.9/33.3 (BP = 1.000 r...,BLEU = 76.41 91.7/82.6/72.7/61.9 (BP = 1.000 r...,BLEU = 40.60 75.0/47.8/31.8/23.8 (BP = 1.000 r...,BLEU = 100.00 100.0/100.0/100.0/100.0 (BP = 1....,BLEU = 63.43 91.7/73.9/63.6/52.4 (BP = 0.920 r...,BLEU = 49.01 87.5/65.2/50.0/33.3 (BP = 0.882 r...,46.667569,76.412541,40.601123,100.0,68.937069,55.533501
2,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,"A Lenovo afirma, que o Yoga 900 tem uma bateri...","A Lenovo afirmar, que até Yoga 900 tem uma bat...","A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo diz que o Yoga 900 tem uma bateria qu...,A Lenovo afirma que o Yoga 900 tem uma pilha q...,0.999628,0.989315,0.951109,1.0,0.975622,0.923828,BLEU = 88.44 95.8/91.3/86.4/81.0 (BP = 1.000 r...,BLEU = 73.29 91.7/82.6/72.7/61.9 (BP = 0.959 r...,BLEU = 31.62 70.8/43.5/22.7/14.3 (BP = 1.000 r...,BLEU = 100.00 100.0/100.0/100.0/100.0 (BP = 1....,BLEU = 60.62 87.5/69.6/59.1/52.4 (BP = 0.920 r...,BLEU = 39.10 83.3/56.5/36.4/19.0 (BP = 0.920 r...,88.438659,76.412541,31.622033,100.0,65.883030,42.499671
3,"A Lenovo afirma, que o Yoga 900 tem uma bateri...","A Lenovo afirma, que o Yoga 900 tem uma bateri...","A Lenovo afirma, que o Yoga 900 tem uma pilhas...","A Lenovo afirmaram, que dos Yoga 900 coloca se...","A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,A Lenovo afirma que o Yoga 900 tem uma bateria...,0.999054,0.955800,0.966651,1.0,0.974700,0.969146,BLEU = 89.12 100.0/95.7/90.9/85.7 (BP = 0.959 ...,BLEU = 73.29 91.7/82.6/72.7/61.9 (BP = 0.959 r...,BLEU = 31.06 70.8/47.8/22.7/14.3 (BP = 0.959 r...,BLEU = 100.00 100.0/100.0/100.0/100.0 (BP = 1....,BLEU = 63.43 91.7/73.9/63.6/52.4 (BP = 0.920 r...,BLEU = 57.21 87.5/65.2/59.1/52.4 (BP = 0.882 r...,92.915644,76.412541,32.384556,100.0,68.937069,64.828561
4,"A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,"A Lenovo afirma, que domínio Yoga 900 tem uma ...","A Lenovo afirma, dela o Yoga 900 tem uma bater...","A Lenovo afirma, que o Yoga 900 tem uma bateri...",A Lenovo afirma que o Yoga 900 tem uma bateria...,A Lenovo afirma que o Yoga 900 tem uma bateria...,0.970302,0.992673,0.936993,1.0,0.975428,0.9714

In [9]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [10]:
from tqdm import tqdm
tqdm.pandas()

In [11]:
original_embedding = model.encode(pira['original'])

In [12]:
original_embedding.shape

(100000, 768)

In [13]:
pira['original_embedding'] = [original_embedding[i] for i in range(original_embedding.shape[0])]

In [14]:
parafrase_embedding = model.encode(pira['parafrase_t5'])
pira['parafrase_t5_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

parafrase_embedding = model.encode(pira['parafrase_sin'])
pira['parafrase_sin_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

parafrase_embedding = model.encode(pira['parafrase_ppdb'])
pira['parafrase_ppdb_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

parafrase_embedding = model.encode(pira['parafrase_frasal'])
pira['parafrase_frasal_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]


parafrase_embedding = model.encode(pira['parafrase_tradutor'])
pira['parafrase_trad_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]


parafrase_embedding = model.encode(pira['parafrase_paranet'])
pira['parafrase_paranet_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

In [15]:
pira['similiaridade_parafrase_t5'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_t5_embedding']).tolist()[0][0], axis=1)

pira['similiaridade_parafrase_sin'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_sin_embedding']).tolist()[0][0], axis=1)

pira['similiaridade_parafrase_ppdb'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_ppdb_embedding']).tolist()[0][0], axis=1)

pira['similiaridade_parafrase_frasal'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_frasal_embedding']).tolist()[0][0], axis=1)


pira['similiaridade_parafrase_trad'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_trad_embedding']).tolist()[0][0], axis=1)


pira['similiaridade_parafrase_paranet'] = pira.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_paranet_embedding']).tolist()[0][0], axis=1)

100%|██████████| 100000/100000 [00:17<00:00, 5847.39it/s]


In [16]:
pira = pira.drop(['index', 'original_embedding', 'parafrase_t5_embedding', 'parafrase_sin_embedding', 'parafrase_ppdb_embedding', 'parafrase_frasal_embedding', 'parafrase_trad_embedding', 'parafrase_paranet_embedding'], axis = 1)
pira.describe()

,similiaridade_parafrase_t5,similiaridade_parafrase_sin,similiaridade_parafrase_ppdb,similiaridade_parafrase_frasal,similiaridade_parafrase_trad,similiaridade_parafrase_paranet
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.940041,0.957060,0.147626,0.142329,0.901638,0.871322
std,0.087345,0.066534,0.226269,0.250625,0.120760,0.131478
min,0.112936,0.230104,-0.262856,-0.263254,0.101261,-0.023981
25%,0.920505,0.942044,0.025156,0.014650,0.870339,0.823805
50%,0.976025,0.985869,0.089035,0.073504,0.946726,0.913799
75%,0.995383,0.999999,0.172191,0.158503,0.981877,0.964494
max,1.000001,1.000002,0.999801,1.000002,1.000001,1.000001


In [17]:
import sacrebleu

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references

pira['BLEUscore_t5'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_t5']]), axis=1)
pira['BLEUscore_sin'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_sin']]), axis=1)
pira['BLEUscore_ppdb'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_ppdb']]), axis=1)
pira['BLEUscore_frasal'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_frasal']]), axis=1)
pira['BLEUscore_trad'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_tradutor']]), axis=1)
pira['BLEUscore_paranet'] = pira.progress_apply(lambda x: sacrebleu.sentence_bleu(x['original'], [x['parafrase_paranet']]), axis=1)

100%|██████████| 100000/100000 [00:37<00:00, 2686.47it/s]


In [18]:
pira['BLEUscore_without_bp_t5'] = pira['BLEUscore_t5'].apply(lambda x: x.score/x.bp)
pira['BLEUscore_without_bp_sin'] = pira['BLEUscore_sin'].apply(lambda x: x.score/x.bp)
pira['BLEUscore_without_bp_ppdb'] = pira['BLEUscore_ppdb'].apply(lambda x: x.score/x.bp)
pira['BLEUscore_without_bp_frasal'] = pira['BLEUscore_frasal'].apply(lambda x: x.score/x.bp)
pira['BLEUscore_without_bp_trad'] = pira['BLEUscore_trad'].apply(lambda x: x.score/x.bp)
pira['BLEUscore_without_bp_paranet'] = pira['BLEUscore_paranet'].apply(lambda x: x.score/x.bp)

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
import nltk
pira['meteor_t5'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_t5'].lower()), axis=1) 
pira['meteor_sin'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_sin'].lower()), axis=1) 
pira['meteor_ppdb'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_ppdb'].lower()), axis=1) 
pira['meteor_frasal'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_frasal'].lower()), axis=1) 
pira['meteor_trad'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_tradutor'].lower()), axis=1) 
pira['meteor_paranet'] =pira.progress_apply(lambda x: nltk.translate.meteor_score.meteor_score([x['original'].lower() for i in range(20)], x['parafrase_paranet'].lower()), axis=1) 

100%|██████████| 100000/100000 [13:19<00:00, 125.03it/s]


In [21]:
pira.describe()

,similiaridade_parafrase_t5,similiaridade_parafrase_sin,similiaridade_parafrase_ppdb,similiaridade_parafrase_frasal,similiaridade_parafrase_trad,similiaridade_parafrase_paranet,BLEUscore_without_bp_t5,BLEUscore_without_bp_sin,BLEUscore_without_bp_ppdb,BLEUscore_without_bp_frasal,BLEUscore_without_bp_trad,BLEUscore_without_bp_paranet
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.940041,0.957060,0.147626,0.142329,0.901638,0.871322,52.246182,78.731209,4.589903,8.992494,31.733021,26.151991
std,0.087345,0.066534,0.226269,0.250625,0.120760,0.131478,27.879714,17.930123,10.012517,24.247310,20.096370,17.436581
min,0.112936,0.230104,-0.262856,-0.263254,0.101261,-0.023981,0.000000,9.375572,0.000000,0.000000,0.000000,0.000000
25%,0.920505,0.942044,0.025156,0.014650,0.870339,0.823805,29.847459,66.750760,0.000000,0.000000,16.233396,12.962473
50%,0.976025,0.985869,0.089035,0.073504,0.946726,0.913799,50.694874,80.465600,2.167132,2.445594,26.269099,21.401603
75%,0.995383,0.999999,0.172191,0.158503,0.981877,0.964494,72.415773,100.000000,3.747777,4.196115,44.285001,35.846689
max,1.000001,1.000002,0.999801,1.000002,1.000001,1.000001,100.000000,100.000000,75.983569,100.000000,100.000000,100.000000


In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
# nltk.download('all')
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in pira['parafrase'].values]
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_vocab)


tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in pira['parafrase'].values]

test_data, _ = padded_everygram_pipeline(n, tokenized_text)


In [ ]:
test_data, _ = padded_everygram_pipeline(n, tokenized_text)

In [ ]:
arq = '/content/drive/My Drive/Colab Notebooks/Pira/Pirá-F-parafraseada_portugues_ptt5_finepira_discriminador.xlsx'
pira.to_excel(arq, index = False)